In [5]:
import pandas as pd

In [6]:
FinalTrainDataNew = pd.read_csv("train_processed2.csv")
FinalTrainDataNew.head(2)

,id,document_text,Accounting and Finance,Antitrust,Banking,Broker Dealer,Commodities Trading,Compliance Management,Consumer protection,Contract Provisions,...,Required Disclosures,Research,Risk Management,Securities Clearing,Securities Issuing,Securities Management,Securities Sales,Securities Settlement,Trade Pricing,Trade Settlement
0,4772,solium financial services llc sfs is a broker ...,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4774,a new year brings new investment opportunities...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

labels = list(FinalTrainDataNew.columns)
y = FinalTrainDataNew[labels[2:]]

tfidf = TfidfVectorizer(analyzer='word', max_features=100000)
X = tfidf.fit_transform(FinalTrainDataNew['document_text'])

# Save the vectorizer
import pickle
pickle.dump(tfidf, open("tfidf.pickle", "wb"))

In [10]:
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingCVClassifier 
from sklearn.multiclass import OneVsRestClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier


clf1 = OneVsRestClassifier(SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None))
clf2 = OneVsRestClassifier(LinearSVC(random_state=0, tol=1e-5))
clf3 = OneVsRestClassifier(LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial'))
clf4 = OneVsRestClassifier(DecisionTreeClassifier(random_state=0))
clf5 = OneVsRestClassifier(RandomForestClassifier(n_estimators=100, random_state=0))

lr = LogisticRegression()
clff = OneVsRestClassifier(lr)
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3, clf4, clf5], meta_classifier=clff)

sclf.fit(X, y)

e:\S\venv\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
e:\S\venv\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
e:\S\venv\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
e:\S\venv\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
e:\S\venv\lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `Tru

StackingClassifier(classifiers=[OneVsRestClassifier(estimator=SGDClassifier(alpha=0.001,
                                                                            max_iter=5,
                                                                            random_state=42,
                                                                            tol=None)),
                                OneVsRestClassifier(estimator=LinearSVC(random_state=0,
                                                                        tol=1e-05)),
                                OneVsRestClassifier(estimator=LogisticRegression(multi_class='multinomial',
                                                                                 random_state=0)),
                                OneVsRestClassifier(estimator=DecisionTreeClassifier(random_state=0)),
                                OneVsRestClassifier(estimator=RandomForestClassifier(random_state=0))],
                   meta_classifier=OneVsRestClassifier(estimator=LogisticRegression()))

In [11]:
import pickle
pickle.dump(sclf, open('SGD_SVC_LR_DT_RF.pkl','wb'))

In [17]:
import pickle
pickle.dump(tfidf, open("tfidf.pickle", "wb"))

In [18]:
model = pickle.load(open('SGD_SVC_LR_DT_RF.pkl','rb'))
tfidf = pickle.load(open("tfidf.pickle", "rb"))

In [19]:
import re

def clean_text(text):
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)

    text = text.split()
    text = ' '.join(text)
    return text

Document = "Settlement, in law, a compromise or agreement between litigants to settle the matters in dispute between them in order to dispose of and conclude their litigation. Generally, as a result of the settlement, prosecution of the action is withdrawn or dismissed without any judgment being entered (see nolle prosequi). In such cases, the settlement itself, as a binding contract between parties, prevents renewal of the litigation. But the parties may, and often do, incorporate the terms of the settlement into a consent judgment, recorded by the court. Such a judgment may afford the same protection against a reopening of the dispute in litigation as is provided by a court judgment at the conclusion of a fully litigated case."

Document = clean_text(Document)

X_test = tfidf.transform([Document])
y_pred = model.predict(X_test)

In [20]:
# Get Labels
labels = list(FinalTrainDataNew.columns)
labels = labels[2:]

# Get Predicted Labels
predicted_labels = []
for i in range(len(y_pred[0])):
    if y_pred[0][i] == 1:
        predicted_labels.append(labels[i])

if len(predicted_labels) == 0:
    predicted_labels = ['Label Not Present']

predicted_labels

['Legal Proceedings', 'Securities Settlement']

In [22]:
labels

['Accounting and Finance',
 'Antitrust',
 'Banking',
 'Broker Dealer',
 'Commodities Trading',
 'Compliance Management',
 'Consumer protection',
 'Contract Provisions',
 'Corporate Communications',
 'Corporate Governance',
 'Definitions',
 'Delivery',
 'Examinations',
 'Exemptions',
 'Fees and Charges',
 'Financial Accounting',
 'Financial Crime',
 'Forms',
 'Fraud',
 'IT Risk',
 'Information Filing',
 'Insurance',
 'Legal',
 'Legal Proceedings',
 'Licensing',
 'Licensure and certification',
 'Liquidity Risk',
 'Listing',
 'Market Abuse',
 'Market Risk',
 'Monetary and Economic Policy',
 'Money Services',
 'Money-Laundering and Terrorist Financing',
 'Natural Disasters',
 'Payments and Settlements',
 'Powers and Duties',
 'Quotation',
 'Records Maintenance',
 'Regulatory Actions',
 'Regulatory Reporting',
 'Required Disclosures',
 'Research',
 'Risk Management',
 'Securities Clearing',
 'Securities Issuing',
 'Securities Management',
 'Securities Sales',
 'Securities Settlement',
 'Tra